In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 777.3 kB/s eta 0:00:0000:0100:14
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 518.9 kB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488511 sha256=5894284e504b3f7a72801a0939f5aca2f79b7ab28f3c4872dd43cd47adb4f7c5
  Stored in directory: /Users/ruthparajo/Library/Caches/pip/wheels/92/09/11/aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
Successfully built pyspark

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark.sql import SparkSession

# Inicializar una sesión de Spark
spark = SparkSession.builder \
    .appName("ParquetToDuckDB") \
    .config("spark.jars.packages", "im.duck:duckdb:0.4.0") \
    .getOrCreate()

# Leer datos desde un archivo Parquet
df = spark.read.parquet("data_lake/compravenda_sup.parquet")

# Mostrar el esquema de los datos
df.printSchema()

# Mostrar las primeras filas de los datos
df.show()

df.write.format("duckdb").option("database", "nombre_de_tu_base_de_datos").option("table", "nombre_de_tu_tabla").mode("overwrite").save()


# Una vez que hayas terminado de trabajar con los datos, es buena práctica cerrar la sesión de Spark
spark.stop()


24/03/22 12:21:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- Any: long (nullable = true)
 |-- Trimestre: long (nullable = true)
 |-- Codi_Districte: long (nullable = true)
 |-- Nom_Districte: string (nullable = true)
 |-- Codi_Barri: long (nullable = true)
 |-- Nom_Barri: string (nullable = true)
 |-- Superfície_mitjana_(m2_construïts): string (nullable = true)
 |-- Nombre: string (nullable = true)

+----+---------+--------------+--------------+----------+--------------------+----------------------------------+------+
| Any|Trimestre|Codi_Districte| Nom_Districte|Codi_Barri|           Nom_Barri|Superfície_mitjana_(m2_construïts)|Nombre|
+----+---------+--------------+--------------+----------+--------------------+----------------------------------+------+
|2023|        1|             1|  Ciutat Vella|         1|            el Raval|                             Total|  63.8|
|2023|        1|             1|  Ciutat Vella|         2|      el Barri Gòtic|                             Total|  82.9|
|2023|        1|             1|  Ciutat Vel

Py4JJavaError: An error occurred while calling o67.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: duckdb. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:248)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Delegati